# Notebook zur Visualisierung der Daten aus der Fragestellung 1.1 und 1.3 im Dashboard
Deises Dasboard von Jupyter bietet die möglichkeit während der Laufzeit die gewünschte Gemeinde auszuwäheln und diese dann als Kuchendiagramm (Pie-Plot) darzustellen. Zusätzlich wird die Heatmap aller Gas-, Heizöl und Elektritzitätsheizungen in einer Karte dargestellt.<br>
Datenquelle sind die aufbereiteten Daten des GWR-Datensatzes die im Modul <1.1_Anteil_fossiler_Heizungen.ipynb> und <1.3_Energiequellen_GHE.ipynb> erzeugt wurden.
<br>
<br>
Quellen: <br>
Plotly Dash, Open-Source Python-Framwork URL: https://plotly.com/dash [Stand: 2.10.2022]<br>
GitHub Jupyter-dash URL: https://github.com/plotly/jupyter-dash [Stand: 2.10.2022 <br>
Leaflet open-source JavaScript library for mobile-friendly interactive maps URL: https://leafletjs.com [Stand: 12.10.2022]<br>
GitHub dash-leaflet URL: https://github.com/thedirtyfew/dash-leaflet [Stand: 2.10.2022]<br>
---
<i> CAS Spatial Data Analytics 2022 </i> ¦ <i> Kommunale Übersicht von Heizsystemen und Energieträgern in Wohngebäuden </i> ¦ <i> Stand: 14.10.2022  </i> ¦ <i> Entwickler: Jürg Reist </i>

### Notebook vorbereiten

In [1]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from ipyleaflet import Map
import dash_leaflet as dl
import dash_leaflet.express as dlx

# für lokal auf Windows
#import dash_core_components as dcc 
#import dash_html_components as html

# für Docker auf Linux
from dash import html, dcc 
from dash.dependencies import Input, Output

from dash_extensions.javascript import assign
from csv import DictReader

stylesheet = ['assets/style.css']

### Lesen alle Heizungen die mit Gas, Heizöl oder Elektrizität geheizt werden
Diese Daten werden im Notebook <1.1_Anteil_fossiler_Heizungen.ipynb> vorbereitet

In [2]:
#lesen der Daten aus dem Modul <1.1_Anteil_fossiler_Heizungen.ipynb>
df = pd.read_csv(r'Export_Data/EnergyProGemeinde_CH.csv')

# Datensatz anzeigen
df

,Unnamed: 0,Gemeinde,Energiequelle,Anzahl
0,0,Zürich,Gas,21658
1,1,Zürich,Heizöl,12714
2,2,Zürich,Keine,12445
3,3,Basel,Fernwärme,9660
4,4,Basel,Gas,8812
...,...,...,...,...
21454,21454,Thurnen,Gas,1
21455,21455,Arogno,Erdwärme,1
21456,21456,Pfungen,Luft,1
21457,21457,Le Bémont (JU),Erdwärme,1


In [3]:
#sortierte Liste aller Gemeinden erstellen. Diese wird im Dash bei der Dropdownliste verwendet
lsGemeinden = sorted(list(df.Gemeinde.unique()))

### Liste aller Gemeinden in der Schweiz mit Koordinaten WGS84 erstellen
So wechselt die Karte automatisch zur ausgewählten Gemeinde, welche in der Dropdownliste ausgewählt wurde.

Info zum Datensatz: https://opendata.swiss/de/dataset/amtliches-ortschaftenverzeichnis-mit-postleitzahl-und-perimeter [Stand: 14.10.2022]<br>
Download: CSV , WGS84 Download: https://data.geo.admin.ch/ch.swisstopo-vd.ortschaftenverzeichnis_plz/PLZO_CSV_WGS84.zip [Stand: 14.10.2022]   

In [4]:
# Lesen aller Gemeinden und speichern in eine list of Dictionaries
with open("GWR_Data/CH_Gemeinden.csv", 'r') as f:
    dict_reader = DictReader(f)
    cities = list(dict_reader)

# Erster Eintrag anzeigen       
cities [0]

{'name': 'Aadorf', 'lat': '47.49135943', 'lon': '8.897698233'}

In [5]:
# Generieren eines geojson Fils mit Maker für jede Gemeinde inkl. dem Namen als Tooltip
geojson = dlx.dicts_to_geojson([{**c, **dict(tooltip=c['name'])} for c in cities])

# Estellen der Javascript Funktion für das Filtern nach eines gewählten Gemeindenamens
geojson_filter = assign("function(feature, context){return context.props.hideout.includes(feature.properties.name);}")

### JupyterDash erstellen und Webserver auf dem Lokalhost mit Prot 8050 starten
Hier wird das Dash vorbereitet... bla bla bla...

gehostet wird das ganze auf einem NAS zu Hause bei Jürg Reist
- Notebook und Dash lauft in einen Docker Container (jupyter/datascience-notebook) https://registry.hub.docker.com/r/jupyter/datascience-notebook/
- Die Karte ist ein WMS aus einem QGIS Server. Auch dieser läuft Docker Container (camptocamp/qgis-server) https://registry.hub.docker.com/r/camptocamp/qgis-server/
- Eigene Sublevel Domains (https://lab.rei.st und https://dash.rei.st) inkl. SSL
- Reversproxy auf NAS
- DNS auf NAS


In [6]:
# JupyterDash erstellen
app = JupyterDash(__name__, external_stylesheets=stylesheet)

#Titel
app.layout = html.Div(children=[
    html.Div(children=[
        html.H3(children='Wie gross ist der Anteil der Heizöl-, Gas- und Elektrizitätsheizungen in einer Gemeinde?'),
        ], style={'textAlign': 'left'}),

    ### Gemeinde Auswahl ###
    html.Div(children=[
        html.Label([
        "Gemeinde auswählen:",
        dcc.Dropdown(id='Gemeinde-dropdown', clearable=False, value=lsGemeinden[0], options=[{'label': c, 'value': c}
                for c in lsGemeinden
            ])
        ])
    ], className="two columns",
       style={'padding':'2rem', 'marginTop':'1rem', 'marginLeft':'1rem', 'boxShadow': '#e3e3e3 4px 4px 2px', 'border-radius': '10px', 'backgroundColor': 'withe'}),        

             
    ### PiePlot für die ausgewählte Gemeinde ###
    html.Div(children=[
             dcc.Graph(id='graph')
        ], className="four columns", style={'padding':'2rem', 'marginTop':'1rem', 'marginLeft':'1rem', 'boxShadow': '#e3e3e3 4px 4px 2px', 'border-radius': '10px', 'backgroundColor': 'withe'}),
    
     
    #### Karte mit WMS vom QGIS-Server und im Hintergund die Geojson Marker####
    html.Div(children=[
           # Definiton des eigenen WMS 
           dl.Map([dl.WMSTileLayer(url="https://wms.rei.st/?SERVICE=WMS", layers="Analyse", format="image/png", transparent=True),
           dl.GeoJSON(data=geojson, options=dict(filter=geojson_filter), id="geojson", zoomToBounds=True)       
           ],
               #center=[47.493, 8.902], zoom=15,
               style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}),    
              
    ], className="six columns", style={'padding':'2rem', 'marginTop':'1rem', 'marginLeft':'1rem', 'boxShadow': '#e3e3e3 4px 4px 2px', 'border-radius': '10px'}),  
             
    
    ### Fusszeile ###
    html.Div(html.P('Weiterbildung der Hochschule für Architektur, Bau und Geomatik FHNW ¦ CAS Spatial Data Analytics 2022 ¦ Kommunale Übersicht von Heizsystemen und Energieträgern in Wohngebäuden ¦ Stand: 12.10.2022 ¦ Student: Jürg Reist'),
            className="eleven columns", style={'textAlign': 'left', 'padding':'1rem', 'marginTop':'5rem', 'marginLeft':'1rem', 'boxShadow': '#e3e3e3 4px 4px 2px', 'border-radius': '10px', 'font-style': 'italic'}
            )
    
], style={'padding': '2rem'})


# Definition der callback Funktion um die Grafik der gerade ausgewählten Gemeinde zu erstellen
@app.callback(
    Output('graph', 'figure'),
    [Input("Gemeinde-dropdown", "value")]
)

def update_figure(Gemeinde):
    dfselect = df.loc[(df['Gemeinde']==Gemeinde)]
    fig = px.pie(dfselect, values='Anzahl', names='Energiequelle', title='<b> Gemeinde ' + Gemeinde + '</b>')
    #image = 'Export_Data/Heatmaps_QGIS/'+ Gemeinde +'.png'
    #image = src=b64_image('Export_Data/Heatmaps_QGIS/'+ Gemeinde +'.png')
    return fig

# Definition der callback Funktion um die Karte der gerade ausgewählten Gemeinde zu aktualisieren
# Dies könnte auch als normaler callback aufegrufen werden. Ist aber so performanter
app.clientside_callback("function(x){return x;}", Output("geojson", "hideout"), Input("Gemeinde-dropdown", "value"))

# Starten der App, um das Resultat in einem externen Browser-Tab anzuzeigen
if __name__ == '__main__':
      app.run_server(mode='external', host='0.0.0.0', port=8050, debug=False)


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8050
 * Running on http://172.17.0.4:8050
Press CTRL+C to quit
127.0.0.1 - - [14/Oct/2022 21:41:06] "GET /_alive_2c545d83-774c-44f9-820a-23e768ceb97a HTTP/1.1" 200 -


Dash app running on http://0.0.0.0:8050/
